# Viewing databse data and filtering to Houston only and Adding Coordinates
* Before running this code:
    * Run the removing trailing spaces sql code within the sql marketing schema
    * This update only needs to be ran once. If you done this already please disregard and continue with the below

In [32]:
import os
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import datetime
import pandas.io.sql as pdsql
from sqlalchemy import create_engine, event, DateTime
from db_setup import mysql_user, mysql_password, db_name
import mysql.connector

In [12]:
connection_string = f"{mysql_user}:{mysql_password}@localhost:3306/{db_name}?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [13]:
engine.table_names()

['ccim',
 'franchise_tax_info',
 'franchise_tax_info_date',
 'ftoffdir',
 'sales_tax_info',
 'sales_tax_info_date']

### Assigning variable to Nominatin()

In [77]:
geolocator = Nominatim(user_agent="data-estate", timeout=None)

## Fran

In [68]:
df_franchise_tax_info_date = pdsql.read_sql("SELECT * FROM franchise_tax_info_date",engine)
df_franchise_tax_info_date.head(10)

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code,Taxpayer_Organizational_Name,Record_Type_Name,DateTime
0,12020053802,"KAGER INDUSTRIES, LLC",3168 N STATE HIGHWAY 16,FREDERICKSBRG,TX,78624,86,CL,8309979391,U,...,2020-01-01 00:00:00,,ROBERT KAGER,3168 NORTH STATE HWY. 16,FREDERICKSBURG,TX,78624,Limited Liability Company - Texas,Secretary of State (SOS) File Number,2020-03-25 02:13:35
1,13415672842,FLAMING RIVER INDUSTRIES INC,800 POERTNER DR,BEREA,OH,44017,0,CF,4408264488,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35
2,13513654320,LA SALLE COMPANY INC,4315 RALPH JONES CT,SOUTH BEND,IN,46628,0,CF,5742717644,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35
3,13636316849,LABSOURCE INC,1260 GARNET DR,NORTHLAKE,IL,60164,0,CF,6304135641,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35
4,14610272230,RUSHING TRANSPORT LLC,7043 E CAPRICE AVE,BATON ROUGE,LA,70811,0,CI,2252688365,X,...,None,,,,,,,Limited Liability Company - Foreign,Comptroller Assigned File Number,2020-03-25 02:13:35
5,16804629406,GAMBA VINEYARDS LOT#18,PO BOX 1124,WINDSOR,CA,95492,0,CF,7075425892,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35
6,17703625081,NETWORK DESIGNS AND INTEGRATION SERVICES,103 HAMMOND AVE,FREMONT,CA,94539,0,CF,5102499549,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35
7,18141771289,INTERSTATE CARRIER LLC,12210 MONTWOOD DR # 103-37,EL PASO,TX,79928,71,CI,2107752596,X,...,None,,,,,,,Limited Liability Company - Foreign,Comptroller Assigned File Number,2020-03-25 02:13:35
8,18243049048,HART FUELING SERVICES LLC,PO BOX 9,NEWTOWN SQ,PA,19073,0,CI,6103312725,X,...,None,,,,,,,Limited Liability Company - Foreign,Comptroller Assigned File Number,2020-03-25 02:13:35
9,19523205292,GAR-BRO MANUFACTURING COMPANY,PO BOX 1077,HEBER SPRINGS,AR,72543,0,CF,5013628171,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35


In [71]:
df_franchise_tax_info_date= df_franchise_tax_info_date[df_franchise_tax_info_date['Taxpayer_City']=='HOUSTON']
df_franchise_tax_info_date.count()

Taxpayer_Number                   12944
Taxpayer_Name                     12944
Taxpayer_Address                  12944
Taxpayer_City                     12944
Taxpayer_State                    12944
Taxpayer_Zip_Code                 12944
Taxpayer_County_Code              12944
Taxpayer_Organizational_Type      12944
Taxpayer_Phone_Number             12944
Record_Type_Code                  12944
Responsibility_Beginning_Date     12944
Secretary_of_State_File_Number    12944
SOS_Charter_Date                  12903
SOS_Status_Date                   12903
Current_Exempt_Reason_Code        12944
Agent_Name                        12944
Agent_Address                     12944
Agent_City                        12944
Agent_State                       12944
Agent_Zip_Code                    12944
Taxpayer_Organizational_Name      12578
Record_Type_Name                  12944
DateTime                          12944
dtype: int64

In [78]:
loc = geolocator.geocode('12210 MONTWOOD DR EL PASO TX')#df1['address'] = 
print(loc.latitude, loc.longitude)

31.760867832666264 -106.26549172632149


In [74]:
df1 = pd.DataFrame()
df1['address'] = df_franchise_tax_info_date['Taxpayer_Address']+" "+df_franchise_tax_info_date['Taxpayer_City']+" "+df_franchise_tax_info_date['Taxpayer_State']
df1.head()

,address
10,3355 W ALABAMA ST STE 630 HOUSTON TX
86,9009 RULAND RD HOUSTON TX
115,2707 KIPLING ST HOUSTON TX
124,3003 SEAGLER RD APT 6203 HOUSTON TX
129,138 TINA LN HOUSTON TX


In [79]:
df1['coordinates']=df1['address'].apply(geolocator.geocode)
df1.head()

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [ ]:
df1['Lat'] = df1['coordinates'].apply(lambda x : x.latitude if x != None else None)
df1['Long'] = df1['coordinates'].apply(lambda x : x.longitude if x != None else None)

In [19]:
df_franchise_tax_info_date = df_franchise_tax_info_date[['Taxpayer_Number','Taxpayer_Name','Taxpayer_Address','Taxpayer_City','Taxpayer_State','Taxpayer_Zip_Code','Taxpayer_Organizational_Type','Taxpayer_Organizational_Name','Record_Type_Code','Record_Type_Name','SOS_Charter_Date','SOS_Status_Date','Responsibility_Beginning_Date','DateTime']]
df_franchise_tax_info_date.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_Organizational_Type,Taxpayer_Organizational_Name,Record_Type_Code,Record_Type_Name,SOS_Charter_Date,SOS_Status_Date,Responsibility_Beginning_Date,DateTime
10,32050048308,"GULF COAST PRE-STRESS, INC.",3355 W ALABAMA ST STE 630,HOUSTON,TX,77098,CT,Texas Profit,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35
86,32072223376,SPRING BRANCH ISD EXPLORE POST 900,9009 RULAND RD,HOUSTON,TX,77055,CN,Texas Non-Profit,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35
115,32072296588,THE LOUIS L. BORICK FOUNDATION,2707 KIPLING ST,HOUSTON,TX,77098,CF,Foreign Profit,X,Comptroller Assigned File Number,None,None,2020-01-02 00:00:00,2020-03-25 02:13:35
124,32072316154,NDN GOLDCRAFT JEWELRIES LLC,3003 SEAGLER RD APT 6203,HOUSTON,TX,77042,CL,Limited Liability Company - Texas,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35
129,32072320818,POWER DESIGN AND ILLUMINATIONS LLC,138 TINA LN,HOUSTON,TX,77037,CL,Limited Liability Company - Texas,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35


In [20]:
formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'HOUfran.csv')
df_franchise_tax_info_date.to_csv(formattedData, index = False)

## STP

In [21]:
df_sales_tax_info_date = pdsql.read_sql("SELECT * FROM sales_tax_info_date",engine)
df_sales_tax_info_date.head()

,Taxpayer_Number,Outlet_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Phone_Number,Outlet_Name,...,Outlet_State,Outlet_Zip_Code,Outlet_County_Code,Outlet_Phone_Number,Permit_Type,Tax_Code,Outlet_NAICS_Code,Permit_Issue_Date,First_Sale_Date,DateTime
0,32072900262,1,JAIBRYN DOMINIQUE AIKENS,4103 BELT LINE RD APT 255,ADDISON,TX,75001,57,5173776337,CENTERFOLD SWIM,...,TX,75001,057,4044683423,SALES TAX,26,454110,2019-12-31 00:00:00,2019-12-31 00:00:00,2020-03-25 02:13:35
1,32012027135,26,"FSG GROUP, L.L.C.",4520 NORTHAVEN RD,DALLAS,TX,75229,57,2146498954,"FSG GROUP, L.L.C.",...,TX,75001,057,9725716328,SALES TAX,26,812112,2019-12-30 00:00:00,2019-12-30 00:00:00,2020-03-25 02:13:35
2,32072770830,1,SNOBALL ONE INC,14681 MIDWAY RD STE 200,ADDISON,TX,75001,57,2142124390,SNOBALL,...,TX,75001,057,2144507076,SALES TAX,26,454110,2020-01-02 00:00:00,2020-01-02 00:00:00,2020-03-25 02:13:35
3,32066046387,2,"TEXADIA HOME TECHNOLOGIES, LLC",4355 EXCEL PKWY STE 600,ADDISON,TX,75001,57,2149565820,"TEXADIA HOME TECHNOLOGIES, LLC",...,TX,75001,057,0 0,SALES TAX,26,238210,2020-01-03 00:00:00,2020-01-03 00:00:00,2020-03-25 02:13:35
4,32052228080,2,STEVE BURKE,4355 EXCEL PKWY STE 600,ADDISON,TX,75001,57,2149565820,TEXADIA SYSTEMS,...,TX,75001,057,0 0,SALES TAX,26,519190,2020-01-03 00:00:00,2020-01-03 00:00:00,2020-03-25 02:13:35


In [22]:
df_sales_tax_info_date= df_sales_tax_info_date[df_sales_tax_info_date['Taxpayer_City']=='HOUSTON']
df_sales_tax_info_date.count()

Taxpayer_Number          5948
Outlet_Number            5948
Taxpayer_Name            5948
Taxpayer_Address         5948
Taxpayer_City            5948
Taxpayer_State           5948
Taxpayer_Zip_Code        5948
Taxpayer_County_Code     5948
Taxpayer_Phone_Number    5948
Outlet_Name              5948
Outlet_Address           5948
Outlet_City              5948
Outlet_State             5948
Outlet_Zip_Code          5948
Outlet_County_Code       5948
Outlet_Phone_Number      5948
Permit_Type              5948
Tax_Code                 5948
Outlet_NAICS_Code        5948
Permit_Issue_Date        5948
First_Sale_Date          5948
DateTime                 5948
dtype: int64

In [23]:
df_sales_tax_info_date = df_sales_tax_info_date[['Taxpayer_Number','Taxpayer_Name','Taxpayer_Address','Taxpayer_City','Taxpayer_State','Taxpayer_Zip_Code','Outlet_Name','Outlet_Address','Outlet_City','Outlet_State','Outlet_Zip_Code','Tax_Code','Permit_Type','Permit_Issue_Date','First_Sale_Date','DateTime']]
df_sales_tax_info_date.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Outlet_Name,Outlet_Address,Outlet_City,Outlet_State,Outlet_Zip_Code,Tax_Code,Permit_Type,Permit_Issue_Date,First_Sale_Date,DateTime
62,32058183693,"PRIMEVOX COMMUNICATIONS, LLC",1321 UPLAND DR # 8245,HOUSTON,TX,77043,PRIMEVOX COMMUNICATIONS,9300 JOHN HICKMAN PKWY STE 1204,FRISCO,TX,75035,26,SALES TAX,2019-12-30 00:00:00,2019-12-30 00:00:00,2020-03-25 02:13:35
164,32069082314,"SEAFOOD ISLAND, INC.",12599 WESTHEIMER RD,HOUSTON,TX,77077,HOT CRAB,714 W SPRING VALLEY RD,RICHARDSON,TX,75080,26,SALES TAX,2020-01-03 00:00:00,2020-01-03 00:00:00,2020-03-25 02:13:35
630,32070926111,"THE SALADO SAWMILL CO, LLC",440 LOUISIANA ST STE 952,HOUSTON,TX,77002,THE SALADO SAWMILL CO.,2330 TAHUAYA DR,BELTON,TX,76513,26,SALES TAX,2020-01-02 00:00:00,2020-01-02 00:00:00,2020-03-25 02:13:35
696,32072900965,JULIAN DAVIS,1414 TEXAS ST UNIT 530,HOUSTON,TX,77002,DISTILLED DESIGNS,1414 TEXAS ST UNIT 530,HOUSTON,TX,77002,26,SALES TAX,2019-12-31 00:00:00,2019-12-31 00:00:00,2020-03-25 02:13:35
697,32072919007,JAMES NGOC LINH NGUYEN,1310 PRAIRIE ST STE 150,HOUSTON,TX,77002,HOUSTON CANDY STORE,1310 PRAIRIE ST STE 150,HOUSTON,TX,77002,26,SALES TAX,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35


In [24]:
formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'HOUstp.csv')
df_sales_tax_info_date.to_csv(formattedData, index = False)

## FTACT

In [25]:
df_franchise_tax_info = pdsql.read_sql("SELECT * FROM  franchise_tax_info WHERE Taxpayer_City = 'HOUSTON';",engine)
df_franchise_tax_info.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name,Rigth_to_Tansact_Business_Name,DateTime
0,10104336481,FLOTATION TECHNOLOGIES INC,945 BUNKER HILL RD STE 500,HOUSTON,TX,77024,101,CF,2072827749,X,...,N,0,None,420000,Foreign Profit,Comptroller Assigned File Number,Active or Inactive with no Reason Code,None,Forfeited,2020-03-25 02:13:35
1,10105480411,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,101,CT,2814440545,U,...,A,0,None,541213,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35
2,10105483688,"CYPRESS CREEK PERSONNEL SERVICES, INC.",13131 CHAMPIONS DR STE 202,HOUSTON,TX,77069,101,CT,2815802180,U,...,A,0,None,561000,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35
3,10105495120,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,101,CT,7139210088,U,...,A,0,None,811121,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35
4,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,101,CT,2816478449,U,...,A,0,None,0,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35


In [26]:
df_franchise_tax_info.count()

Taxpayer_Number                   508136
Taxpayer_Name                     508134
Taxpayer_Address                  508136
Taxpayer_City                     508136
Taxpayer_State                    508136
Taxpayer_Zip_Code                 508136
Taxpayer_County_Code              508136
Taxpayer_Organizational_Type      508136
Taxpayer_Phone_Number             295324
Record_Type_Code                  508136
Responsibility_Beginning_Date     508136
Responsibility_End_Date                0
Responsibility_End_Reason_Code    508136
Secretary_of_State_File_Number    508136
SOS_Charter_Date                  493660
SOS_Status_Date                   493660
SOS_Status_Code                   493660
Rigth_to_Tansact_Business_Code    508136
Current_Exempt_Reason_Code        508136
Exempt_Begin_Date                  34772
NAICS_Code                        508136
Taxpayer_Organizational_Name      508136
Record_Type_Name                  508136
Responsibility_End_Reason_Name    508136
SOS_Status_Name 

In [27]:
df_franchise_tax_info = df_franchise_tax_info[['Taxpayer_Number','Taxpayer_Name','Taxpayer_Address','Taxpayer_City','Taxpayer_State','Taxpayer_Zip_Code','Taxpayer_Organizational_Type','Taxpayer_Organizational_Name','Record_Type_Code','Record_Type_Name','Responsibility_Beginning_Date','Responsibility_End_Reason_Code','Responsibility_End_Reason_Name','SOS_Charter_Date','SOS_Status_Date','SOS_Status_Code','SOS_Status_Name','Rigth_to_Tansact_Business_Code','Rigth_to_Tansact_Business_Name','DateTime']]
df_franchise_tax_info.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_Organizational_Type,Taxpayer_Organizational_Name,Record_Type_Code,Record_Type_Name,Responsibility_Beginning_Date,Responsibility_End_Reason_Code,Responsibility_End_Reason_Name,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,SOS_Status_Name,Rigth_to_Tansact_Business_Code,Rigth_to_Tansact_Business_Name,DateTime
0,10104336481,FLOTATION TECHNOLOGIES INC,945 BUNKER HILL RD STE 500,HOUSTON,TX,77024,CF,Foreign Profit,X,Comptroller Assigned File Number,2000-12-01 00:00:00,0,Active or Inactive with no Reason Code,None,None,None,None,N,Forfeited,2020-03-25 02:13:35
1,10105480411,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-12-17 00:00:00,0,Active or Inactive with no Reason Code,2001-12-17 00:00:00,2001-12-17 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
2,10105483688,"CYPRESS CREEK PERSONNEL SERVICES, INC.",13131 CHAMPIONS DR STE 202,HOUSTON,TX,77069,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-11-20 00:00:00,0,Active or Inactive with no Reason Code,2001-11-20 00:00:00,2001-11-20 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
3,10105495120,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-12-04 00:00:00,0,Active or Inactive with no Reason Code,2001-12-04 00:00:00,2001-12-04 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
4,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,CT,Texas Profit,U,Secretary of State (SOS) File Number,2007-01-01 00:00:00,0,Active or Inactive with no Reason Code,2012-12-22 00:00:00,2012-12-22 00:00:00,A,Active,A,Active,2020-03-25 02:13:35


In [28]:
formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'HOUFTACT.csv')
df_franchise_tax_info.to_csv(formattedData, index = False)

## STACT

In [29]:
df_sales_tax_info = pdsql.read_sql("SELECT * FROM  sales_tax_info WHERE Taxpayer_City = 'HOUSTON';", engine)
df_sales_tax_info.head()

,Taxpayer_Number,Outlet_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Phone_Number,Taxpayer_Organizational_Type,...,Outlet_State,Outlet_Zip_Code,Outlet_County_Code,Outlet_Phone_Number,Outlet_NAICS_Code,Outlet_Inside_Outside_City_Limits_Indicator,Outlet_Permit_Issue_Date,Outlet_First_Sales_Date,Taxpayer_Organizational_Name,DateTime
0,10105480411,1,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,101,2814440545,CT,...,TX,77090,101,None,441228,N,2002-01-25 00:00:00,2002-03-01 00:00:00,TEXAS PROFIT CORPORATION,2020-03-25 02:13:35
1,10105495120,1,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,101,7139210088,CT,...,TX,77011,101,None,811121,Y,2003-03-10 00:00:00,2002-01-03 00:00:00,TEXAS PROFIT CORPORATION,2020-03-25 02:13:35
2,10105517915,3,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,101,2816478449,CT,...,TX,77084,101,None,541330,Y,2017-10-18 00:00:00,2017-09-01 00:00:00,TEXAS PROFIT CORPORATION,2020-03-25 02:13:35
3,10105517915,4,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,101,0,CT,...,TX,77531,20,None,541330,Y,2018-01-22 00:00:00,2018-01-01 00:00:00,TEXAS PROFIT CORPORATION,2020-03-25 02:13:35
4,10105547839,2,"S S S ENTERPRISES, INC.",8901 HOWARD DR,HOUSTON,TX,77017,101,8327152371,CT,...,TX,77017,101,None,445120,Y,2005-12-01 00:00:00,2005-12-02 00:00:00,TEXAS PROFIT CORPORATION,2020-03-25 02:13:35


In [30]:
df_sales_tax_info = df_sales_tax_info[['Taxpayer_Number','Taxpayer_Name','Taxpayer_Address','Taxpayer_City','Taxpayer_State','Taxpayer_Zip_Code','Outlet_Name','Outlet_Address','Outlet_City','Outlet_State','Outlet_Zip_Code','Taxpayer_Organizational_Type','Taxpayer_Organizational_Name','Outlet_Permit_Issue_Date','Outlet_First_Sales_Date','DateTime']]
df_sales_tax_info.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Outlet_Name,Outlet_Address,Outlet_City,Outlet_State,Outlet_Zip_Code,Taxpayer_Organizational_Type,Taxpayer_Organizational_Name,Outlet_Permit_Issue_Date,Outlet_First_Sales_Date,DateTime
0,10105480411,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,MOTORCYCLES UNLIMITED,1964 FM 1960 RD W,HOUSTON,TX,77090,CT,TEXAS PROFIT CORPORATION,2002-01-25 00:00:00,2002-03-01 00:00:00,2020-03-25 02:13:35
1,10105495120,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,LIL IKES INC,7110 AVENUE C,HOUSTON,TX,77011,CT,TEXAS PROFIT CORPORATION,2003-03-10 00:00:00,2002-01-03 00:00:00,2020-03-25 02:13:35
2,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,CT,TEXAS PROFIT CORPORATION,2017-10-18 00:00:00,2017-09-01 00:00:00,2020-03-25 02:13:35
3,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,"WOOD GROUP USA, INC.",622 COMMERCE ST,CLUTE,TX,77531,CT,TEXAS PROFIT CORPORATION,2018-01-22 00:00:00,2018-01-01 00:00:00,2020-03-25 02:13:35
4,10105547839,"S S S ENTERPRISES, INC.",8901 HOWARD DR,HOUSTON,TX,77017,ON THE GO MART,8901 HOWARD DR,HOUSTON,TX,77017,CT,TEXAS PROFIT CORPORATION,2005-12-01 00:00:00,2005-12-02 00:00:00,2020-03-25 02:13:35


In [31]:
formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'HOUSTACT.csv')
df_sales_tax_info.to_csv(formattedData, index = False)